<style>
    body { 
        width: 960px;
        margin: 0 auto;
    }
    
    h1, h2, .main-header, .header-text {
        text-align: center;
    }
    
    .widget-gridbox {
        margin: 0 auto;
    }
    
    .scale {
        border: 1px solid #ddd;
        border-radius: 4px;
        display: block;
        margin-left: auto !important;
        margin-right: auto !important;
        width: 40% !important;
}
</style>

<h1 class="main-header">Évaluateur de la qualité nutritionnelle</h1>

<!-- <div class="header-text">
Voudriez-vous savoir si votre produit alimentaire est sain ?<br> 
Renseignez les valeurs nutritionnelles de votre produit</div> -->

In [652]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Voila app imports
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import GridBox, AppLayout, Button, Layout

# Read image data from URL
from PIL import Image
import requests
from io import BytesIO

In [351]:
# Load result.csv
df = pd.read_csv('result.csv')

In [405]:
required_facts = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g', 'salt_100g']
optional_facts = ['fiber_100g', 'cholesterol_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 'trans-fat_100g', 'additives_n', 'ingredients_from_palm_oil_n']
mineral_facts = ['potassium_100g', 'calcium_100g', 'phosphorus_100g', 'magnesium_100g']
vitamins = ['vitamin-a_100g', 'vitamin-d_100g', 'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g', 'vitamin-pp_100g', 'vitamin-b6_100g', 'vitamin-b9_100g', 'vitamin-b12_100g']

In [566]:
# Split the data into train and test sets
# This lets us simulate how our model will perform in the future
xtrain, xtest, ytrain, ytest = train_test_split(df[required_facts + optional_facts + vitamins + mineral_facts], df[["nutrition-score-fr_100g"]], test_size=0.2, random_state=42)
# Instantiate a classifier and train it
model = LinearRegression()
model.fit(xtrain, ytrain)
# Evaluate the model's performance
#model.score(xtrain, ytrain)
print("")

In [563]:
#model.score(xtest, ytest)

In [565]:
# Make predictions
# predictions = model.predict(xtest)
# predictions

In [671]:
def results(energy, fat, sat_fat, carbs, sugars, proteins, salt, fiber, cholesterol, mono_fat, poly_fat, trans_fat, additives, palm_oil, 
            v_a, v_d, v_c, v_b1, v_b2, v_pp, v_b6, v_b9, v_b12, potassium, calcium, phosphorus, magnesium):
    nutriments = [energy, fat, sat_fat, carbs, sugars, proteins, 
                  salt, fiber, cholesterol, mono_fat, poly_fat, 
                  trans_fat, additives, palm_oil, v_a, v_d, v_c, 
                  v_b1, v_b2, v_pp, v_b6, v_b9, v_b12, potassium, 
                  calcium, phosphorus, magnesium
                 ]
    
    newdf = pd.DataFrame([nutriments], 
                         columns = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 
                                    'proteins_100g', 'salt_100g','fiber_100g', 'cholesterol_100g', 'monounsaturated-fat_100g', 
                                    'polyunsaturated-fat_100g', 'trans-fat_100g', 'additives_n', 'ingredients_from_palm_oil_n',
                                    'vitamin-a_100g', 'vitamin-d_100g', 'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g', 
                                    'vitamin-pp_100g', 'vitamin-b6_100g', 'vitamin-b9_100g', 'vitamin-b12_100g', 
                                    'potassium_100g', 'calcium_100g', 'phosphorus_100g', 'magnesium_100g' 
                                   ]
    
                   )
    
    [[prediction]] = model.predict(newdf)
    #prediction_final = prediction
    
    img_link = get_image_link(prediction) if sum(nutriments) != 0 else get_image_link()
    
    prediction_html = f"""
        <img src={img_link} class="scale">
    """
    #{round(prediction_final, 2):,}
    
    display(HTML(prediction_html))
    
def get_image_link(score=np.nan):
    links = {'affreux': 'https://i.ibb.co/ZYyns8z/affreux.png',
             'mauvais': 'https://i.ibb.co/FqbTfKD/mauvais.png',
             'mediocre': 'https://i.ibb.co/b1QtfpJ/mediocre.png',
             'bon': 'https://i.ibb.co/GVSHwBn/bon.png',
             'excellent': 'https://i.ibb.co/qJ76Mc6/excellent.png', 
             'start': 'https://i.ibb.co/wRZ97JT/start.png'
            }
        
    if score < 0: return links['excellent']
    elif score >= 0 and score <= 2: return links['bon']
    elif score > 2 and score <= 10: return links['mediocre']
    elif score > 10 and score <= 18: return links['mauvais']
    elif score > 18: return links['affreux']
    else: return links['start']

In [393]:
# For testing
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

In [588]:
# Widgets - required facts
form_item_layout_m = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    width='250px', 
    overflow='visible'
)

style = {'description_width': '150px'}
style_additives = {'description_width': '170px'}
style_fats = {'description_width': '150px', 'text_color': 'white', 'background': '#DDAA00'}

energy_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=999999,
    step=0.1,
    description='Énergie (kJ) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

fat_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='Matières grasses (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

sat_fat_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='dont saturés (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m,
)

carbs_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='Glucides (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

sugars_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='dont sucres (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

proteins_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='Protéines (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

salt_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='Sel (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

In [532]:
# Widgets - optional facts
fiber_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='dont fibres (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

cholesterol_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='Cholestérol (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

mono_fat_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='dont mono-insaturés (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

poly_fat_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='dont poly-insaturés (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

trans_fat_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100.0,
    step=0.1,
    description='dont trans (g) :',
    style=style,
    disabled=False,
    layout=form_item_layout_m
)

additives_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100,
    step=1,
    description="Additifs :",
    style=style_additives,
    disabled=False,
)

palm_oil_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100,
    step=1,
    description="Dérivés de l'huile de palme :",
    style=style_additives,
    disabled=False
)

In [577]:
# Widgets - vitamins
v_a_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="A :",
    disabled=False,
    layout=form_item_layout_m
)

v_d_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="D :",
    disabled=False,
    layout=form_item_layout_m
)

v_c_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="C :",
    disabled=False,
    layout=form_item_layout_m
)

v_b1_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="B1 :",
    disabled=False,
    layout=form_item_layout_m
)

v_b2_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="B2 :",
    disabled=False,
    layout=form_item_layout_m
)

v_pp_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="PP :",
    disabled=False,
    layout=form_item_layout_m
)

v_b6_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="B6 :",
    disabled=False,
    layout=form_item_layout_m
)

v_b9_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="B9 :",
    disabled=False,
    layout=form_item_layout_m
)

v_b12_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="B12 :",
    disabled=False,
    layout=form_item_layout_m
)

In [530]:
# Widgets - minerals
potassium_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="Potassium :",
    disabled=False,
    layout=form_item_layout_m
)

calcium_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="Calcium :",
    disabled=False,
    layout=form_item_layout_m
)

phosphorus_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="Phosphore :",
    disabled=False,
    layout=form_item_layout_m
)

magnesium_widget = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.01,
    description="Magnésium :",
    disabled=False,
    layout=form_item_layout_m
)

In [589]:
# Widgets - html elements
header_label = widgets.HTML(value='<h2>Valeurs nutritionnelles pour 100 g</h2>')

hr = widgets.HTML(value='<hr>')

In [556]:
# Grouped widgets
vitamin_widgets = [v_a_widget, v_d_widget, v_c_widget, v_b1_widget, v_b2_widget, v_pp_widget, v_b6_widget, v_b9_widget, v_b12_widget]
mineral_widgets = [potassium_widget, calcium_widget, phosphorus_widget, magnesium_widget]
additive_widgets = [additives_widget, palm_oil_widget]

list_widgets = [widgets.VBox(vitamin_widgets),
                widgets.VBox(mineral_widgets), 
                widgets.VBox(additive_widgets)
                ]

accordion = widgets.Accordion(children=list_widgets, titles=('Vitamines (g)', 'Minéraux (g)', 'Additifs (nombre)'))

In [416]:
out2 = widgets.interactive_output(results, {'energy': energy_widget, 
                                            'fat': fat_widget, 
                                            'sat_fat': sat_fat_widget, 
                                            'carbs': carbs_widget, 
                                            'sugars': sugars_widget, 
                                            'proteins': proteins_widget, 
                                            'salt': salt_widget,
                                            'fiber': fiber_widget,
                                            'cholesterol': cholesterol_widget,
                                            'mono_fat': mono_fat_widget,
                                            'poly_fat': poly_fat_widget,
                                            'trans_fat': trans_fat_widget,
                                            'additives': additives_widget,
                                            'palm_oil': palm_oil_widget,
                                            'v_a': v_a_widget,
                                            'v_d': v_d_widget,
                                            'v_c': v_c_widget,
                                            'v_b1': v_b1_widget,
                                            'v_b2': v_b2_widget,
                                            'v_pp': v_pp_widget,
                                            'v_b6': v_b6_widget,
                                            'v_b9': v_b9_widget,
                                            'v_b12': v_b12_widget,
                                            'potassium': potassium_widget,
                                            'calcium': calcium_widget,
                                            'phosphorus': phosphorus_widget,
                                            'magnesium': magnesium_widget
                                           })

In [592]:
vb = widgets.VBox()
vb.children = [energy_widget,
               hr,
               fat_widget,
               sat_fat_widget,
               poly_fat_widget,
               mono_fat_widget,
               trans_fat_widget,
               hr,
               carbs_widget,
               sugars_widget,
               fiber_widget,
               hr,
               proteins_widget,
               hr,
               cholesterol_widget,
               hr,
               salt_widget
              ]

In [667]:
app = AppLayout(header=header_label,
                left_sidebar=vb,
                center=None,
                right_sidebar=accordion,
                footer=None,
                pane_widths=[3, 1, "350px"],
                pane_heights=[1, 5, 1]
               )

In [590]:
gb = widgets.HBox()
gb.children = [app]

In [655]:
display(gb)

In [663]:
display(out2)

Output(layout=Layout(grid_area='footer'))